# Assignment 8
author: Rachel Nelson

class: DSC650

#### Assignment 8
For this assignment, we will be using data from the Berkeley Deep Drive. We will only use a small fraction of the original dataset as the full dataset contains hundreds of gigabytes of video and other data. In particular, this assignment uses route data to simulate data collected from GPS and accelerometer sensors within a car. The data has already been pre-processed in a format and structure that is easy to use with Spark Streaming.

The data/processed/bdd/ folder contains the processed data for this assignment. The accelerations folder contains accelerometer data collected from each car and the locations contain the GPS data. Each folder contains sub-folders organized by the timestamp of the simulation.

In [1]:
import os
import json
import time
from collections import namedtuple
import heapq
import uuid
import pandas as pd
import s3fs
import pyarrow.parquet as pq

endpoint_url='https://storage.budsc.midwest-datascience.com'
s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': endpoint_url
    }
)

acceleration_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'x',
    'y',
    'z',
#     't'
]
Acceleration = namedtuple('Acceleration', acceleration_columns)
def read_accelerations():
    df = pq.ParquetDataset(
        's3://data/processed/bdd/accelerations',
        filesystem=s3
    ).read_pandas().to_pandas()
    
    df = df[acceleration_columns].sort_values(by=['offset'])
    
    records = [Acceleration(*record) for record in df.to_records(index=False)]
    
    return records
accelerations = read_accelerations()

location_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'course',
    'latitude',
    'longitude',
    'geohash',
    'speed',
    'accuracy',
#     't'
]
Location = namedtuple('Location', location_columns)
def read_locations():
    df = pq.ParquetDataset(
        's3://data/processed/bdd/locations',
        filesystem=s3
    ).read_pandas().to_pandas()
    
    df = df[location_columns].sort_values(by=['offset'])
    
    records = [Location(*record) for record in df.to_records(index=False)]
    
    return records
    
locations = read_locations()

In [17]:
json_location = json.dumps(locations)
json_acccelerations = json.dumps(accelerations)

In [18]:
import json
import uuid

from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin.new_topic import NewTopic
from kafka.errors import TopicAlreadyExistsError

import os
import pandas as pd
import datetime
import threading
from pathlib import Path
import time
import s3fs
import pyarrow.parquet as pq
from collections import namedtuple


In [19]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Rachel',
    last_name='Nelson'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Rachel',
 'last_name': 'Nelson',
 'client_id': 'NelsonRachel',
 'topic_prefix': 'NelsonRachel'}

In [20]:
def loadParquet(parq_path):
    pqr = spark.read.parquet(parq_path)
    # Convert from spark dataframe to pandas dataframe
    pqr = pqr.toPandas()
    return pqr

def splitstr(std):
    before, after = str(std).split('.')
    return before, after

def startTimer(results_dir):
    # Loop on time
    print("call function here")
    retval = startTimedParquetStreamUpdateLoop(results_dir)
    # Stop if time is over and there are no more partitions.
    if ((time.time() - start_time) < 70 and retval == 0):
        t = threading.Timer(interval, startTimer(results_dir))

In [21]:
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))

In [22]:
create_kafka_topic('locations')
create_kafka_topic('accelerations')

Topic "NelsonRachel-locations" already exists
Topic "NelsonRachel-accelerations" already exists


In [23]:
producer = KafkaProducer(
  bootstrap_servers=config['bootstrap_servers'],
  value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

In [24]:
def on_send_success(record_metadata):
    print('Message sent:\n    Topic: "{}"\n    Partition: {}\n    Offset: {}'.format(
        record_metadata.topic,
        record_metadata.partition,
        record_metadata.offset
    ))
    
def on_send_error(excp):
    print('I am an errback', exc_info=excp)
    # handle exception

def send_data(topic, data, config=config, producer=producer, msg_key=None):
    topic_prefix = config['topic_prefix']
    topic_name = '{}-{}'.format(topic_prefix, topic)
    
    if msg_key is not None:
        key = msg_key
    else:
        key = uuid.uuid4().hex
    
    producer.send(
        topic_name, 
        value=data,
        key=key.encode('utf-8')
    ).add_callback(on_send_success).add_errback(on_send_error)

In [25]:
example_data = dict(
    key1='value1',
    key2='value2'
)

send_data('locations', example_data)
send_data('locations',json_location)

Message sent:
    Topic: "NelsonRachel-locations"
    Partition: 0
    Offset: 11
Message sent:
    Topic: "NelsonRachel-locations"
    Partition: 0
    Offset: 12


In [26]:
import json

from kafka import KafkaConsumer

In [27]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Rachel',
    last_name='Nelson'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config


{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Rachel',
 'last_name': 'Nelson',
 'client_id': 'NelsonRachel',
 'topic_prefix': 'NelsonRachel'}

In [28]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

In [29]:
general_consumer.topics()

{'ChougulePushkar-accelerations',
 'ChougulePushkar-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'GonzalezKimberly-accelerations',
 'GonzalezKimberly-joined',
 'GonzalezKimberly-locations',
 'GonzalezKimberly-simple',
 'GonzalezKimberly-windowed',
 'HoffmannLaura-accelerations',
 'HoffmannLaura-locations',
 'KaakatyRoni-accelerations',
 'KaakatyRoni-locations',
 'KalaikkovanVasanthakumar-accelerations',
 'KalaikkovanVasanthakumar-locations',
 'KanaparthiVenkata-accelerations',
 'KanaparthiVenkata-locations',
 'KohnAaron-accelerations',
 'KohnAaron-locations',
 'KolekarShilpa-accelerations',
 'KolekarShilpa-locations',
 'Last_Name_testFirst_Name_test-locations',
 'MandalAbhijit-accelerations',
 'MandalAbhijit-locations',
 'MohammedShabuddinRamizuddin-accelerations',
 'MohammedShabuddinRamizuddin-locations',
 'MuleyTushar-accelerations',
 'MuleyTushar-accelrations',
 'MuleyTushar-locations',
 'MuthusamyGaneshkumar-accelerations',
 'MuthusamyGaneshkumar-

In [30]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

In [31]:
consumer.subscription()

{'NelsonRachel-accelerations', 'NelsonRachel-locations'}

In [32]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 0
Message Key: 8d4d1848c6124b078ae7d0798f65e12a
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 1
Message Key: 7857e36d21134f5fbd834c5c12c73adf
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 2
Message Key: 171193d2ae9e4eb899bf5d89181238f1
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 3
Message Key: c4f170df0be0444aa8e0a3eae52e3688
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations
  Partition: 0
  Offset: 4
Message Key: 4cef1228d1464f439dfab20c868512b4
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: NelsonRachel-locations


STOPPING MESSAGE CONSUMER


In [33]:

consumer.close()